### Exploratory Data Analysis
 - Perform Exploratory data analysis
 - Filter outliers 
 - Write to a delta table

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# get variables from environment
dbutils.widgets.text("catalog_name", "bms")
dbutils.widgets.text("schema_name", "occupancy_project")

catalog_name = dbutils.widgets.get("catalog_name")
schema_name = dbutils.widgets.get("schema_name")

In [ ]:
# read silver table data
df = spark.read.table(f"{catalog_name}.{schema_name}.silverTable")
df = df.toPandas()
df.head()

In [ ]:
# explore general statistics of dataset
df.describe()

In [ ]:
# cast data types
# define data type
data_types = {
    "Temperature": float,
    "Humidity": float,
    "Light": float,
    "CO2": float,
    "HumidityRatio": float,
    "Occupancy": int
}

# apply to the dataframe
df = df.astype(data_types)
# convert the date data type
df["date"] = pd.to_datetime(df["date"])

In [ ]:
# dataset info
df.info(), df.shape

In [ ]:
# class distribution of the target label
sns.catplot(data=df, x="Occupancy", kind="count")

##### Distribution plots of data features

In [ ]:
# distribution of features
sns.displot(df, x="Temperature", hue="Occupancy")

In [ ]:
# distribution plot for CO2 value
sns.displot(data=df, x="CO2", hue="Occupancy", kind="kde")

In [ ]:
# distribution plot for Light value
sns.displot(data=df, x="Light", hue="Occupancy", kind="kde")

In [ ]:
# distribution plot for Humidity value
sns.displot(data=df, x="Humidity", hue="Occupancy", kind="kde")

In [ ]:
# distribution plot for Humidity value
sns.displot(data=df, x="HumidityRatio", hue="Occupancy", kind="kde")

In [ ]:
# Box plot of CO2
sns.catplot(data=df, x="Occupancy", y="CO2", kind="box")

In [ ]:
# box plot of Temperature
sns.catplot(data=df, x="Occupancy", y="Temperature", kind="box")

In [ ]:
# box plot of Light
sns.catplot(data=df, x="Occupancy", y="Light", kind="box")

In [ ]:
# box plot of Humidity
sns.catplot(data=df, x="Occupancy", y="Humidity", kind="box")

In [ ]:
# boxplot of humidity ratio
sns.catplot(data=df, x="Occupancy", y="HumidityRatio", kind="box")

In [ ]:
# feature correlation 
sns.heatmap(df.corr(), annot=True)

#### Remove Outliers

In [ ]:
# filter outliers
# This method is not perfect, but it's a good starting point
unoccupied_df = df[
    (df["Temperature"] < 22) & 
    (df["CO2"] < 720) & 
    (df["Occupancy"] == 0)
][["Temperature", "Humidity", "Light", "CO2", "HumidityRatio", "Occupancy"]]

occupied_df = df[
    (df["Temperature"] > 21) & 
    (df["Temperature"] < 23.1) & 
    (df["CO2"] < 1550) & 
    (df["Occupancy"] == 1)
][["Temperature", "Humidity", "Light", "CO2", "HumidityRatio", "Occupancy"]]
combined_df = pd.concat([unoccupied_df, occupied_df])
combined_df.reset_index(drop=True, inplace=True)

In [ ]:
# Write data to delta gold table
spark.createDataFrame(combined_df).write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.goldtable")